In [1]:
import tensorflow as tf
import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
AUTOTUNE = tf.data.experimental.AUTOTUNE
import multiprocessing
import numpy as np
from IPython.display import clear_output
num_cpus = multiprocessing.cpu_count()
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU') 
tf.config.experimental.set_visible_devices(devices=my_devices, device_type='CPU')
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

2023-10-01 12:39:31.489478: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-01 12:39:32.135759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-01 12:39:32.800253: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 12:39:32.823452: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=4100)]
    )
logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
cpus = tf.config.list_physical_devices('CPU')
logical_cpus = tf.config.list_logical_devices('CPU')
print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPUs")

1 Physical GPU, 1 Logical GPUs
1 Physical CPU, 1 Logical CPUs


2023-10-01 12:39:33.435891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 12:39:33.436221: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 12:39:33.436420: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
path_inp = "../data_for_gan/input"
path_out = "../data_for_gan/output"
input = []
output1 = []
output2 = []
for root, dirs, files in os.walk(path_inp):
    for file in files:
        input.append(os.path.join(root, file))
        output1.append(path_out+"/"+file[0]+"/"+file[0]+"1.png")
        output2.append(path_out+"/"+file[0]+"/"+file[0]+"2.png")

In [4]:
input, output1, output2

(['../data_for_gan/input/4.jpg',
  '../data_for_gan/input/3.jpg',
  '../data_for_gan/input/5.png',
  '../data_for_gan/input/1.jpg',
  '../data_for_gan/input/6.jpg'],
 ['../data_for_gan/output/4/41.png',
  '../data_for_gan/output/3/31.png',
  '../data_for_gan/output/5/51.png',
  '../data_for_gan/output/1/11.png',
  '../data_for_gan/output/6/61.png'],
 ['../data_for_gan/output/4/42.png',
  '../data_for_gan/output/3/32.png',
  '../data_for_gan/output/5/52.png',
  '../data_for_gan/output/1/12.png',
  '../data_for_gan/output/6/62.png'])

In [5]:
LR = 0.001
# Custom loss function
def bce_dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)  # Привести y_true к типу float32
    # dice_loss = dice_coef(y_true, y_pred)
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return bce_loss

def dice_coef(y_true, y_pred, smooth=1e-5):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)
    return (2.0 * intersection + smooth) / (union + smooth)

NUM_EPOCHS=25
IMG_WIDTH       = 512
IMG_HEIGHT      = 512
IMG_CHANNELS    = 3

In [37]:
def convolution_block(
    block_input,
    num_filters=128,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = tf.keras.layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding=padding,
        use_bias=use_bias,
        kernel_initializer=tf.keras.initializers.HeNormal(),
    )(block_input)
    x = tf.keras.layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    dspp_input = tf.keras.layers.BatchNormalization()(dspp_input)
    x = tf.keras.layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = tf.keras.layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="nearest" # nearest
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = tf.keras.layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

def DeeplabV3Plus(input_shape, num_classes):
    model_input = tf.keras.Input(shape=input_shape)
    resnet50 = tf.keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    # print(len(resnet50.layers))
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = tf.keras.layers.UpSampling2D(
        size=(input_shape[0] // 4 // x.shape[1], input_shape[0] // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = tf.keras.layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = convolution_block(x)
    x = tf.keras.layers.UpSampling2D(
        size=(input_shape[0] // x.shape[1], input_shape[0] // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same", activation="sigmoid")(x)
    return tf.keras.Model(inputs=model_input, outputs=model_output)
model = DeeplabV3Plus((IMG_HEIGHT, IMG_WIDTH, 3), 3)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3*1e-4), loss=tf.keras.losses.CategoricalCrossentropy())

In [38]:
@tf.function
def load_image_and_mask(image_path, mask1_path, mask2_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH), method="bicubic")
    image = tf.cast(image, dtype=tf.float32) / 255.
    mask1 = tf.io.read_file(mask1_path)
    mask1 = tf.image.decode_png(mask1, channels=1)
    mask1 = tf.image.resize(mask1, (IMG_HEIGHT, IMG_WIDTH))
    mask1 = ~tf.cast(mask1, dtype=tf.uint8)
    mask2 = tf.io.read_file(mask2_path)
    mask2 = tf.image.decode_png(mask2, channels=1)
    mask2 = tf.image.resize(mask2, (IMG_HEIGHT, IMG_WIDTH))
    mask2 = ~tf.cast(mask2, dtype=tf.uint8)
    mask1 = tf.expand_dims(mask1, axis=0)  # Добавить размерность спереди
    mask2 = tf.expand_dims(mask2, axis=0)  # Добавить размерность спереди
    mask1 = tf.squeeze(mask1, axis=-1)
    mask2 = tf.squeeze(mask2, axis=-1)
    mask = tf.stack([(~mask1 & ~mask2), mask1, mask2], axis=-1)
    image = tf.expand_dims(image, axis=0)
    return image, mask

In [39]:
batch_size = 3
input_dataset = tf.data.Dataset.from_tensor_slices(input)
output1_dataset = tf.data.Dataset.from_tensor_slices(output1)
output2_dataset = tf.data.Dataset.from_tensor_slices(output2)

# Примените load_image_and_mask к каждому элементу внутри датасетов
# input_dataset = input_dataset.map(load_image_and_mask, num_parallel_calls=AUTOTUNE)
# output1_dataset = output1_dataset.map(load_image_and_mask, num_parallel_calls=AUTOTUNE)
# output2_dataset = output2_dataset.map(load_image_and_mask, num_parallel_calls=AUTOTUNE)

# Зипуйте датасеты вместе
dataset = tf.data.Dataset.zip((input_dataset, output1_dataset, output2_dataset))
dataset_train = dataset.map(load_image_and_mask, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [40]:
import matplotlib.pyplot as plt
for x,y in dataset_train:
    print(x.shape, y.shape)
    # print(dat[0].shape, dat[1].shape, )

(1, 512, 512, 3) (1, 512, 512, 3)
(1, 512, 512, 3) (1, 512, 512, 3)
(1, 512, 512, 3) (1, 512, 512, 3)
(1, 512, 512, 3) (1, 512, 512, 3)
(1, 512, 512, 3) (1, 512, 512, 3)


In [41]:
history = model.fit(dataset_train, epochs=100)

Epoch 1/100


2023-10-01 12:52:45.505382: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_6/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


5/5 [==============================] - 21s 155ms/step - loss: 194.0169
Epoch 2/100
5/5 [==============================] - 1s 148ms/step - loss: 120.8947
Epoch 3/100
5/5 [==============================] - 1s 149ms/step - loss: 81.5569
Epoch 4/100
5/5 [==============================] - 1s 171ms/step - loss: 70.5469
Epoch 5/100
5/5 [==============================] - 2s 411ms/step - loss: 66.8408
Epoch 6/100
5/5 [==============================] - 1s 149ms/step - loss: 58.7479
Epoch 7/100
5/5 [==============================] - 1s 149ms/step - loss: 51.7719
Epoch 8/100
5/5 [==============================] - 1s 148ms/step - loss: 46.3028
Epoch 9/100
5/5 [==============================] - 1s 153ms/step - loss: 42.0498
Epoch 10/100
5/5 [==============================] - 2s 435ms/step - loss: 38.7689
Epoch 11/100
5/5 [==============================] - 1s 149ms/step - loss: 36.6146
Epoch 12/100
5/5 [==============================] - 1s 149ms/step - loss: 34.9484
Epoch 13/100
5/5 [================

In [13]:
model.save("mod", save_format='tf')

INFO:tensorflow:Assets written to: mod/assets


INFO:tensorflow:Assets written to: mod/assets
